multi query retriver tries to solve the ambiguity in the user's query by generating the multiple queries out of it

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [3]:
# initialize the embeddings
embedding_model = HuggingFaceEmbeddings()

# create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

c:\Users\dheka\OneDrive\Desktop\11.Langchain_from_beginning\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# create retrievers
similarity_retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [8]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k" : 5}),
    llm = ChatHuggingFace(
        llm = HuggingFaceEndpoint(
            repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
            task="text-generation",
            huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")
        )
    )
)

In [9]:
# Query
query = "how to maintain energy levels and maintain balance?"

In [10]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [11]:
for i, doc in enumerate(similarity_results):
    print(f"\n----Result {i+1} -----")
    print(doc.page_content)

print("\n")
print("*" * 50)
print("\n")

for i, doc in enumerate(multiquery_results):
    print(f"\n----Result {i+1} -----")
    print(doc.page_content)


----Result 1 -----
Drinking sufficient water throughout the day helps maintain metabolism and energy.

----Result 2 -----
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

----Result 3 -----
Regular walking boosts heart health and can reduce symptoms of depression.

----Result 4 -----
Consuming leafy greens and fruits helps detox the body and improve longevity.

----Result 5 -----
The solar energy system in modern homes helps balance electricity demand.


**************************************************



----Result 1 -----
Drinking sufficient water throughout the day helps maintain metabolism and energy.

----Result 2 -----
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

----Result 3 -----
Consuming leafy greens and fruits helps detox the body and improve longevity.

----Result 4 -----
Regular walking boosts heart health and can reduce symptoms of depression.

----Result 5 -----
Deep sleep is crucial for cellular repa